In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import (
    ColumnTransformer,
    TransformedTargetRegressor,
    make_column_transformer,
)

# Load data

In [2]:
data = pd.read_csv('../data/cleaned_data_v2.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 33 columns):
 #   Column                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                         --------------  -----  
 0   case number                                                                                                                                                                                                                    696 non-null    object 
 1   Who was the member adjudicating the decision?                                                                                         

In [3]:
data = data.rename(columns={
    'What was the length of the tenancy, or in other words, how long had the tenants lived at the residence in question? '
    :'What was the length of the tenancy',
    'What was the amount of the rental deposit? ':'rental deposit amount',
    'If any rent increases occurred, what was the rent after the increase(s)?':'was there an rent increases',
    'Over how many months did the arrears accumulate? ':'Over how many months did the arrears accumulate?',
    'If the tenant made a payment on the arrears after the eviction notice was served and/or prior to the hearing, what was the amount of the payment? ':'Does the tenant made a payment on the arrears after the eviction notice',
    'What were the specific mental, medical, or physical conditions of the tenant, if any? ':'What were the specific mental, medical, or physical conditions of the tenant, if any?'})

In [4]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=123)
X_train, y_train = train_df.drop(columns = ["What was the outcome of the case?"]), train_df["What was the outcome of the case?"]
X_test, y_test = test_df.drop(columns = ["What was the outcome of the case?"]), test_df["What was the outcome of the case?"]
X_train.head()

,case number,Who was the member adjudicating the decision?,What was the location of the landlord tenant board?,Did the decision state the landlord was represented?,Did the decision state the landlord attended the hearing?,Did the decision state the tenant was represented?,Did the decision state the tenant attended the hearing?,Did the decision state the landlord was a not-for-profit landlord (e.g. Toronto Community Housing)?,Did the decision state the tenant was collecting a subsidy?,What was the length of the tenancy,...,"If the tenant was employed, did the decision state any doubts about the stability of employment e.g. lack of guaranteed hours, contract work, etc.?",Did the member find the tenant had sufficient income to pay rent?,Did the decision mention the tenant lost their job leading up to or during the period of the hearing?,"Did the decision mention any other extenuating circumstances experienced by the tenant leading up to or during the period of the claim (e.g. hospitalization, death in the family, etc.)?","Did the decision mention the tenant’s difficulty finding alternative housing for any reason e.g.physical limitations, reliance on social assistance, etc.?",Did the decision state the tenant was given prior notice for the eviction?,Did the decisions state postponement would result in the tenant accruing additional arrears?,Which other specific applications of the landlord or the tenant were mentioned?,Did the decision mention the validity of an N4 eviction notice?,Payment Plan
195,TSL-90833-17,Roger Rodrigues,Toronto,1,0,0,0,0,0,6.306049,...,0,1,0,0,0,0,0,[],1,0.0
553,TNL-07861-18,Nancy Morris,Toronto,0,1,0,1,0,0,24.000000,...,1,0,0,1,0,1,0,[],0,0.5
598,TSL-96267-18,David Mungovan,Toronto,1,0,0,1,0,0,6.306049,...,0,0,0,0,0,1,0,[],0,0.0
645,TEL-81094-17,Shelby Whittick,Whitby,0,1,0,1,0,0,6.306049,...,0,0,0,1,0,1,1,"['L1', 'L2']",0,0.0
634,TEL-80073-17,Jim McMaster,Toronto,1,1,0,1,0,0,6.306049,...,0,0,0,0,0,1,1,"['L1', 'L2', 'T2']",0,0.0


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 195 to 510
Data columns (total 32 columns):
 #   Column                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                         --------------  -----  
 0   case number                                                                                                                                                                                                                    556 non-null    object 
 1   Who was the member adjudicating the decision?                                                                                       

In [6]:
y_train.value_counts()

No relief                   352
Postponement of eviction    124
Payment plan                 64
Relief                       12
Conditional Order             4
Name: What was the outcome of the case?, dtype: int64

## Preprocess data with pipeline

In [7]:
categorical_features = ['Who was the member adjudicating the decision?',
                        'What was the location of the landlord tenant board?',
                        'Which other specific applications of the landlord or the tenant were mentioned?'
                        ]

numeric_features = [
'What was the length of the tenancy',
'What was the monthly rent?',
'rental deposit amount',
'What was the total amount of arrears?',
'Over how many months did the arrears accumulate?',

]
ordinal_features = [
'Was the tenant employed at the time of the hearing?',
'If the tenant was not employed, did the decision state the tenant was receiving any form of government assistance (e.g. OW, childcare benefits, ODSP, OSAP)?',
'If the tenant was employed, did the decision state any doubts about the stability of employment e.g. lack of guaranteed hours, contract work, etc.?',
'Payment Plan'
]
drop_features = ['case number']
ordering_ordinal = [
    [-1,0,1],
    [-1,0,1],
    [-1,0,1],
    [0, 0.5, 1]
]

In [8]:
preprocessor = make_column_transformer(
        (OneHotEncoder(handle_unknown="ignore"),categorical_features),
        (StandardScaler(),numeric_features),
        (OrdinalEncoder(categories = ordering_ordinal),ordinal_features),
        ("drop",drop_features))
preprocessor

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Who was the member adjudicating the '
                                  'decision?',
                                  'What was the location of the landlord '
                                  'tenant board?',
                                  'Which other specific applications of the '
                                  'landlord or the tenant were mentioned?']),
                                ('standardscaler', StandardScaler(),
                                 ['What was the length of the tenancy',
                                  'What was the monthly rent?',
                                  'r...
                                 ['Was the tenant employed at the time of the '
                                  'hearing?',
                                  'If the tenant was not employed, did the '
                                  'decision state the tenant was receiving any '
                                  'form of government assistance (e.g. OW, '
                                  'childcare benefits, ODSP, OSAP)?',
                                  'If the tenant was employed, did the '
                                  'decision state any doubts about the '
                                  'stability of employment e.g. lack of '
                                  'guaranteed hours, contract work, etc.?',
                                  'Payment Plan']),
                                ('drop', 'drop', ['case number'])])

## Baseline model

In [9]:
# adapt from DSCI573 lecture note
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [10]:
from sklearn.dummy import DummyClassifier

results_dict = {}
scoring_metrics = ["accuracy"]

dummy = make_pipeline(preprocessor, DummyClassifier(random_state = 123))
results_dict["dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, scoring = scoring_metrics, return_train_score=True
)

results_df = pd.DataFrame(results_dict).T
results_df

/Users/AliceYuuu/opt/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.010 (+/- 0.003),0.006 (+/- 0.001),0.633 (+/- 0.004),0.633 (+/- 0.001)


_With no tuned hyperparameters of DummyClassifier, the mean cross validation score of our baseline model is 0.633. Since it is a a multi-class classification and there is class imbalance, this accuracy score is low and not quite representative._

# LightGBM model

In [11]:
from lightgbm.sklearn import LGBMClassifier

pipe_lgbm = make_pipeline(preprocessor, LGBMClassifier(random_state=123))

results_dict["lgbm"] = mean_std_cross_val_scores(
    pipe_lgbm, X_train, y_train, scoring = scoring_metrics, return_train_score=True
)

results_df = pd.DataFrame(results_dict).T
results_df

/Users/AliceYuuu/opt/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.010 (+/- 0.003),0.006 (+/- 0.001),0.633 (+/- 0.004),0.633 (+/- 0.001)
lgbm,0.167 (+/- 0.018),0.008 (+/- 0.001),0.678 (+/- 0.039),0.998 (+/- 0.002)


With default hyperparameters num_leaves = 31 and learning_rate = 0.1, our LightGBM model gets a better mean cross validation score (0.678) than the score of base model (0.663). However, we can see the model is very overfit, with the mean train score being 0.32 higher than the mean cv score. 

# Hyperparameter optimization

In [12]:
param_grid = {
    "lgbmclassifier__learning_rate": np.arange(0.01, 0.2, 0.001),
    "lgbmclassifier__num_leaves": np.arange(10, 80, 10),
    "lgbmclassifier__class_weight": ["balanced", None]
}

random_search = RandomizedSearchCV(
    pipe_lgbm, 
    param_grid, 
    n_jobs=-1, 
    n_iter=20, 
    random_state=123,
    scoring=scoring_metrics, 
    refit='accuracy',
    return_train_score=True
)

random_search.fit(X_train, y_train)

results = pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_accuracy",
        "param_lgbmclassifier__learning_rate",
        "param_lgbmclassifier__num_leaves",
        "param_lgbmclassifier__class_weight",
        "mean_fit_time",
        "rank_test_accuracy",
    ]
].set_index("rank_test_accuracy").sort_index().T
results

/Users/AliceYuuu/opt/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


rank_test_accuracy,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
mean_test_accuracy,0.721252,0.717632,0.699598,0.694192,0.683414,0.681628,0.681596,0.68158,0.679762,0.676207,0.674437,0.672587,0.672555,0.670785,0.668999,0.654633,0.638465,0.631274,0.629472,0.607915
param_lgbmclassifier__learning_rate,0.012,0.014,0.044,0.047,0.172,0.071,0.17,0.083,0.144,0.127,0.072,0.173,0.199,0.144,0.128,0.166,0.025,0.02,0.023,0.016
param_lgbmclassifier__num_leaves,30,60,10,50,50,50,30,70,50,60,30,70,40,50,60,10,70,40,60,60
param_lgbmclassifier__class_weight,None,None,None,None,None,None,balanced,None,None,None,None,balanced,None,balanced,None,balanced,balanced,balanced,balanced,balanced
mean_fit_time,0.284605,0.270844,0.13827,0.266405,0.260712,0.270163,0.264863,0.248703,0.250168,0.277679,0.269332,0.251496,0.216358,0.263708,0.281811,0.157149,0.254861,0.254627,0.257412,0.279277


In [13]:
best_pipe_lgbm = make_pipeline(
    preprocessor, 
    LGBMClassifier(num_leaves = 30, learning_rate = 0.012, class_weight=None, random_state=123)
)

In [14]:
results_dict["best_lgbm"] = mean_std_cross_val_scores(
    best_pipe_lgbm, X_train, y_train, scoring = scoring_metrics, return_train_score=True
)

results_df = pd.DataFrame(results_dict).T
results_df

/Users/AliceYuuu/opt/miniconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.010 (+/- 0.003),0.006 (+/- 0.001),0.633 (+/- 0.004),0.633 (+/- 0.001)
lgbm,0.167 (+/- 0.018),0.008 (+/- 0.001),0.678 (+/- 0.039),0.998 (+/- 0.002)
best_lgbm,0.157 (+/- 0.013),0.009 (+/- 0.001),0.721 (+/- 0.017),0.793 (+/- 0.007)


_As shown above, We tuned three hyperparameters: learning_rate, num_leaves and class_weight. The highest mean cv score we got is 0.721 with learning_rate=0.012, num_leaves=30 and class_weight=None. This is better than with the default hyperparameter leaning_rate=0.1 and num_leaves=31, which has a mean cv score of 0.678. Additionally, we can see that the model is much less overfit with the mean train score being 0.072 higher than the mean cv score._

# Interpretation and feature importances 

In [15]:
#!pip install eli5

In [16]:
import eli5
# eli5 feature importance weights for LGBM classifier
categorical_feature_names = (
    random_search.best_estimator_
    .named_steps["columntransformer"]
    .named_transformers_["onehotencoder"]
    .get_feature_names_out(categorical_features)
    .tolist()
)

feature_names = (
    categorical_feature_names + numeric_features + ordinal_features
)

best_pipe_lgbm.fit(X_train, y_train)

eli5.explain_weights(
    best_pipe_lgbm.named_steps["lgbmclassifier"], feature_names=feature_names
)

Weight,Feature
0.3932,Payment Plan
0.1804,What was the total amount of arrears?
0.0842,Over how many months did the arrears accumulate?
0.0667,rental deposit amount
0.0484,What was the monthly rent?
0.0434,What was the length of the tenancy
0.0415,"If the tenant was not employed, did the decision state the tenant was receiving any form of government assistance (e.g. OW, childcare benefits, ODSP, OSAP)?"
0.0367,"If the tenant was employed, did the decision state any doubts about the stability of employment e.g. lack of guaranteed hours, contract work, etc.?"
0.0249,Was the tenant employed at the time of the hearing?
0.0244,Which other specific applications of the landlord or the tenant were mentioned?_[]


_Based on the eli5 results above, it seems that "Payment Plan" is the strongest variable used by the LGBM Classifiers with a value of 0.3932. The second strongest variable "What was the total amount of arrears?" gets a weight of 0.1804. All other variables have a weight lower than 0.09, which indicates they are not as important._

# Conclusion

In [17]:
test_score = best_pipe_lgbm.score(X_test, y_test)
test_score

0.7

_The test score for my LightGBM model is 0.7, which agrees with the mean validation score of 0.721 from before. Since the validation score and the test score are similar, I don't think there is optimization bias. If there is optimization bias, the validation score would be much higher than the test score. Therefore, I have decent trust in my model, and believe it would do well to deployment data._